<a href="https://colab.research.google.com/github/micaelCZ/Paper_Repositorio/blob/main/Copia_de_A%3FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization, Dropout, Dense, Flatten
import urllib

# Descargar el archivo CSV
url = 'https://github.com/jonathan-elian-toapanta/INTERNETWORKING/raw/main/ESCENARIOS/ScenarioA.csv'
filename = 'ScenarioA.csv'
urllib.request.urlretrieve(url, filename)

# Leer el archivo CSV
dataframe = pd.read_csv(filename, low_memory=False, sep=',')

     



In [5]:
#Normalize
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df
     

dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()

     

keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys) - 1]].copy()
x_normalised = dfNormalize(data_to_process)
print(x_normalised.describe())
change_labels = lambda x: 1 if x == 'nonTOR' else 0
     

y_normalised = dataframe['label'].apply(change_labels)

     

X_train, X_test, y_train, y_test = train_test_split(x_normalised,
                                            y_normalised, test_size=0.3, random_state=42)

           Protocol   Flow Duration   Flow Bytes/s   Flow Packets/s  \
count  67834.000000    67834.000000        67830.0          67828.0   
mean       0.560663        0.299188            0.0              0.0   
std        0.496310        0.406300            0.0              0.0   
min        0.000000        0.000000            0.0              0.0   
25%        0.000000        0.004436            0.0              0.0   
50%        1.000000        0.041086            0.0              0.0   
75%        1.000000        0.732555            0.0              0.0   
max        1.000000        1.000000            0.0              0.0   

        Flow IAT Mean   Flow IAT Std   Flow IAT Max   Flow IAT Min  \
count    67834.000000   67834.000000   67834.000000   67834.000000   
mean         0.031600       0.031363       0.089855       0.019491   
std          0.069971       0.090973       0.173880       0.057865   
min          0.000000       0.000000       0.000000       0.000000   
25%       

In [6]:
import numpy as np

# Verificar si hay valores nulos en x_train e y_train
print(np.isnan(X_train).any())
print(np.isnan(y_train).any())

# Reemplazar valores nulos por cero
x_train = np.nan_to_num(X_train, nan=0.0)
y_train = np.nan_to_num(y_train, nan=0.0)

# Definir el modelo de la CNN
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(x_train.shape[1], 1)))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compilar el modelo
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
# Crear el modelo
model = create_model()

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Hacer predicciones sobre los datos de prueba
y_pred = model.predict(X_test)

# Convertir las probabilidades en etiquetas binarias (0 o 1) usando un umbral de 0.5
y_pred_binary = (y_pred > 0.5).astype(int)
 
# calcular métricas de evaluación
acc = accuracy_score(y_test, y_pred_binary)*100
f1 = f1_score(y_test, y_pred_binary, average='weighted')*100
prec = precision_score(y_test, y_pred_binary, average='weighted')*100
rec = recall_score(y_test, y_pred_binary, average='weighted')*100

print('Accuracy: %.2f' % acc)
print('F1 score: %.2f' % f1)
print('Precision: %.2f' % prec)
print('Recall: %.2f' % rec)

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Reemplazar valores nulos por cero
x_test = np.nan_to_num(X_test, nan=0.0)
y_test = np.nan_to_num(y_test, nan=0.0)

# Predice las probabilidades de las clases para los datos de prueba
y_pred_prob = model.predict(x_test)

# Calcula la curva ROC y el área bajo la curva (AUC)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)



 Protocol          False
 Flow Duration     False
 Flow Bytes/s       True
 Flow Packets/s     True
 Flow IAT Mean     False
 Flow IAT Std      False
 Flow IAT Max      False
 Flow IAT Min      False
Fwd IAT Mean       False
 Fwd IAT Std       False
 Fwd IAT Max       False
 Fwd IAT Min       False
Bwd IAT Mean       False
 Bwd IAT Std       False
 Bwd IAT Max       False
 Bwd IAT Min       False
Active Mean        False
 Active Std        False
 Active Max        False
 Active Min        False
Idle Mean          False
 Idle Std          False
 Idle Max          False
 Idle Min          False
dtype: bool
False
Epoch 1/10
1484/1484 [==============================] - 31s 19ms/step - loss: 0.4835 - accuracy: 0.8840 - val_loss: 0.4083 - val_accuracy: 0.8808
Epoch 2/10
1484/1484 [==============================] - 19s 13ms/step - loss: 0.3803 - accuracy: 0.8817 - val_loss: 0.3684 - val_accuracy: 0.8808
Epoch 3/10
1484/1484 [==============================] - 19s 13ms/step - loss: 0.3646 - acc

C:\Users\Holger\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 88.08
F1 score: 82.50
Precision: 77.59
Recall: 88.08
636/636 [==============================] - 3s 4ms/step
